In [7]:
import requests
from bs4 import BeautifulSoup

# from google.colab import files

fuel_cons = ([40.0, 32.8, 53.0, 84.5, 72.7])

url_req = (['?imei=350424060069364&s_dt=2022-10-01%2006:50:24&e_dt=2022-10-01%2015:41:34',
            '?imei=350424060069364&s_dt=2022-10-11%2007:07:56&e_dt=2022-10-11%2017:46:18',
            '?imei=350424060069364&s_dt=2022-10-15%2006:47:09&e_dt=2022-10-15%2017:08:30',
            '?imei=350424060069364&s_dt=2022-10-17%2007:08:54&e_dt=2022-10-17%2016:42:52',
            '?imei=350424060069364&s_dt=2022-10-18%2006:50:12&e_dt=2022-10-18%2017:34:39'])

web_link = 'https://nav.by/info/testAccel.php'

db_files = []

### postman
import requests

url = "https://nav.by/login_get_data.php?type=LOGIN&data%5Blogin%5D=diler_el_vek&data%5Bpwd%5D=111111&data%5BscreenWidth%5D=1920&data%5BscreenHeight%5D=1080"

payload = {}
headers = {
    'Cookie': 'pc_id=60b9bbc0-bef7-4c4b-9052-003672c5d1a9; ses_id=15fb4b00-046b-4e59-bd88-175381dd03b8'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


### postman

def remove_str(value, deletechars_):
    for c in deletechars_:
        value = value.replace(c, '_')
    return value


NUM_OF_FILES = len(url_req)
for p in range(NUM_OF_FILES):

    name_of_file = remove_str(url_req[p], '\/:*?"<>|&%=-')
    db_files.append(name_of_file)

    print(name_of_file)

    file = open(name_of_file + ".csv", "w")
    file.close()

    response = requests.request("GET", web_link + url_req[p], headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'lxml')

    file = open(name_of_file + ".csv", "a+")

    quotes = soup.find_all('p')
    for quote in quotes:
        file.write(quote.text)
    file.close()

    file = open(name_of_file + ".csv", "r")
    lines = file.readlines()
    file.close()

    f_first_line = 1

    file = open(name_of_file + ".csv", "w")
    for line in lines:
        if f_first_line == 1:
            file.write(line)
            f_first_line = 0
        if ((
                line != "date;speed;78;64011;64022;64033;64044;delta64011;delta64022;delta64033;delta64044;vodo;lat;lon;dir;alt;engine_speed;distance_gps;201;202;203;204;210;211;212;213;214;215;ain1;ain2;151;146;147\n") and (
                line != "\n")):
            file.write(line)

    file.close()

{"url":".\/info\/help.php","exitCode":0}
_imei_350424060069364_s_dt_2022_10_01_2006_50_24_e_dt_2022_10_01_2015_41_34
_imei_350424060069364_s_dt_2022_10_11_2007_07_56_e_dt_2022_10_11_2017_46_18
_imei_350424060069364_s_dt_2022_10_15_2006_47_09_e_dt_2022_10_15_2017_08_30
_imei_350424060069364_s_dt_2022_10_17_2007_08_54_e_dt_2022_10_17_2016_42_52
_imei_350424060069364_s_dt_2022_10_18_2006_50_12_e_dt_2022_10_18_2017_34_39


In [8]:
import math
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
from array import array

sns.set(style="darkgrid")

In [9]:
from scipy.optimize import curve_fit

# initial guesses for k1, k2, k3, k4, k5, k6:
kfc_init = 1.0, 1.0, 1.0, 1.0

def func_fuel_consumption(x, k1, k2, k3, k4):
    delta_es_app_, delta_es_app_3_, delta_es_app_2_, delta_es_ = x
    return k1 * delta_es_app_ + k2 * delta_es_app_3_ + k3 * delta_es_app_2_ + k4 * delta_es_

def func3_fuel_consumption(x, k2, k3, k4, k5):
    delta_es_app_, delta_es_2_app_, delta_es_app_2_, delta_es_2_app_2_ = x
    return k2 * delta_es_app_ + k3 * delta_es_2_app_ + k4 * delta_es_app_2_ + k5 * delta_es_2_app_2_


In [10]:
NUM_OF_FILES = len(db_files)

num_points = [0] * NUM_OF_FILES

delta_ES = []
delta_ES_APP = []
delta_ES_2_APP = []
delta_ES_APP_2 = []
delta_ES_2_APP_2 = []
delta_ES_APP_3 = []
delta_WT = []

delta_level_dut = []

popt_calc = []
popt3_calc = []
popt_calc2 = []
matrix_linear = []

for p in range(NUM_OF_FILES):
    df = pd.read_csv(db_files[p] + '.csv', sep=';', engine='python', decimal='.')
    df.columns = ['date', 'speed', 'rowdata', 'z', 'y', 'x_m', 'x_p', 'delta64011', 'delta64022', 'delta64033',
                  'delta64044', 'vodo', 'lat', 'lon', 'dir', 'alt', 'engine_speed', 'distance_gps', 'sum_ES',
                  'mark_ind1', 'sum_ES_APP', 'mark_ind2', 'sum_ES_2_APP', 'mark_ind3', 'sum_ES_APP_2', 'mark_ind4',
                  'sum_ES_2_APP_2', 'mark_ind5', 'AnIn1', 'AnIn2', 'CAN1', 'CAN2', 'CAN3']

    print('len(' + db_files[p] + '):' + str(len(df)))

    sum_ES_sum = 0
    sum_ES_APP_sum = 0
    sum_ES_2_APP_sum = 0
    sum_ES_APP_2_sum = 0
    sum_ES_2_APP_2_sum = 0
    sum_ES_APP_3_sum = 0
    sum_WT_sum = 0

    sum_ES_inst = 0
    sum_ES_APP_inst = 0
    sum_ES_2_APP_inst = 0
    sum_ES_APP_2_inst = 0
    sum_ES_2_APP_2_inst = 0
    sum_ES_APP_3_inst = 0

    for i in range(2, len(df)):
    #Data from CAN-bus:
        try:
            can1_ = int(df.CAN1[i])
            can1_prev = int(df.CAN1[i-1])
        except:
            print("err1:")
            print(i)
            print(df.CAN1[i])

        sum_ES_inst = (can1_&0xffff) - (can1_prev&0xffff)
        while sum_ES_inst < 0:
            sum_ES_inst += 32000
        sum_ES_sum += sum_ES_inst
        if sum_ES_inst != 0:
            d_time = (datetime.strptime(df.date[i], "%Y-%m-%d %H:%M:%S.%f") - datetime.strptime(df.date[i-1], "%Y-%m-%d %H:%M:%S.%f")).seconds
            sum_WT_sum += d_time

        if ((can1_>>32)&0xffff) >= ((can1_prev>>32)&0xffff):
            sum_ES_APP_inst = ((can1_>>32)&0xffff) - ((can1_prev>>32)&0xffff)
        else:
            sum_ES_APP_inst = ((can1_>>32)&0xffff) - ((can1_prev>>32)&0xffff) + 32000
        sum_ES_APP_sum += sum_ES_APP_inst

        if ((can1_>>48)&0xffff) >= ((can1_prev>>48)&0xffff):
            sum_ES_2_APP_inst = ((can1_>>48)&0xffff) - ((can1_prev>>48)&0xffff)
        else:
            sum_ES_2_APP_inst = ((can1_>>48)&0xffff) - ((can1_prev>>48)&0xffff) +32000
        sum_ES_2_APP_sum += sum_ES_2_APP_inst

        try:
            can2_ = int(df.CAN2[i])
            can2_prev = int(df.CAN2[i-1])
        except:
            print("err2:")
            print(i)
            print(df.CAN2[i])

        if (can2_&0xffff) >= (can2_prev&0xffff):
            sum_ES_APP_2_inst = (can2_&0xffff) - (can2_prev&0xffff)
        else:
            sum_ES_APP_2_inst = (can2_&0xffff) - (can2_prev&0xffff) + 32000
        sum_ES_APP_2_sum += sum_ES_APP_2_inst

        if ((can2_>>16)&0xffff) >= ((can2_prev>>16)&0xffff):
            sum_ES_2_APP_2_inst = ((can2_>>16)&0xffff) - ((can2_prev>>16)&0xffff)
        else:
            sum_ES_2_APP_2_inst = ((can2_>>16)&0xffff) - ((can2_prev>>16)&0xffff) + 32000
        sum_ES_2_APP_2_sum += sum_ES_2_APP_2_inst

        if ((can2_>>32)&0xffff) >= ((can2_prev>>32)&0xffff):
            sum_ES_APP_3_inst = ((can2_>>32)&0xffff) - ((can2_prev>>32)&0xffff)
        else:
            sum_ES_APP_3_inst = ((can2_>>32)&0xffff) - ((can2_prev>>32)&0xffff) +32000
        sum_ES_APP_3_sum += sum_ES_APP_3_inst

#         #Data from RS485:
#         if df.sum_ES[i] > df.sum_ES[i-1]:
#             sum_ES_inst = df.sum_ES[i] - df.sum_ES[i-1]
#             sum_ES_sum += sum_ES_inst
#
#         if df.sum_ES_APP[i] > df.sum_ES_APP[i-1]:
#             sum_ES_APP_inst = df.sum_ES_APP[i] - df.sum_ES_APP[i-1]
#             sum_ES_APP_sum += sum_ES_APP_inst
#
#         if df.sum_ES_2_APP[i] > df.sum_ES_2_APP[i-1]:
#             sum_ES_2_APP_inst = df.sum_ES_2_APP[i] - df.sum_ES_2_APP[i-1]
#             sum_ES_2_APP_sum += sum_ES_2_APP_inst
#
#         if df.sum_ES_APP_2[i] > df.sum_ES_APP_2[i-1]:
#             sum_ES_APP_2_inst = df.sum_ES_APP_2[i] - df.sum_ES_APP_2[i-1]
#             sum_ES_APP_2_sum += sum_ES_APP_2_inst
#
#         if df.sum_ES_2_APP_2[i] > df.sum_ES_2_APP_2[i-1]:
#             sum_ES_2_APP_2_inst = df.sum_ES_2_APP_2[i] - df.sum_ES_2_APP_2[i-1]
#             sum_ES_2_APP_2_sum += sum_ES_2_APP_2_inst

    delta_ES.append(sum_ES_sum)
    delta_ES_APP.append(sum_ES_APP_sum)
    delta_ES_2_APP.append(sum_ES_2_APP_sum)
    delta_ES_APP_2.append(sum_ES_APP_2_sum)
    delta_ES_2_APP_2.append(sum_ES_2_APP_2_sum)
    delta_ES_APP_3.append(sum_ES_APP_3_sum)
    delta_WT.append(sum_WT_sum)

    matrix_linear.append([sum_ES_APP_sum, sum_ES_2_APP_sum, sum_ES_APP_2_sum, sum_ES_2_APP_2_sum])
    delta_level_dut.append(fuel_cons[p])

    if p == 3:
        popt_calc2 = np.linalg.solve(np.array(matrix_linear), np.array(delta_level_dut))
        print("interpolation result(popt_calc2):")
        print(popt_calc2)

print("delta_WT:")
print(delta_WT)
temp_rel = []
for el in delta_WT:
    temp_rel.append(el/delta_WT[0])
print(temp_rel)

print("delta_ES:")
print(delta_ES)
temp_rel = []
for el in delta_ES:
    temp_rel.append(el/delta_ES[0])
print(temp_rel)

print("delta_ES_APP:")
print(delta_ES_APP)
temp_rel = []
for el in delta_ES_APP:
    temp_rel.append(el/delta_ES_APP[0])
print(temp_rel)

popt, pcov = curve_fit(func_fuel_consumption,
                       (delta_ES_APP, delta_ES_APP_3, delta_ES_APP_2, delta_ES),
                       delta_level_dut, kfc_init)
popt_calc = popt
print("interpolation result(popt_calc):")
print(popt_calc)

popt, pcov = curve_fit(func3_fuel_consumption,
                       (delta_ES_APP, delta_ES_2_APP, delta_ES_APP_2, delta_ES_2_APP_2),
                       delta_level_dut, kfc_init)
popt3_calc = popt
print("interpolation result(popt3_calc):")
print(popt3_calc)


len(_imei_350424060069364_s_dt_2022_10_01_2006_50_24_e_dt_2022_10_01_2015_41_34):11094
len(_imei_350424060069364_s_dt_2022_10_11_2007_07_56_e_dt_2022_10_11_2017_46_18):13729
len(_imei_350424060069364_s_dt_2022_10_15_2006_47_09_e_dt_2022_10_15_2017_08_30):13461
len(_imei_350424060069364_s_dt_2022_10_17_2007_08_54_e_dt_2022_10_17_2016_42_52):12046
interpolation result(popt_calc2):
[ 0.00026048 -0.00043941 -0.00034164  0.00072723]
len(_imei_350424060069364_s_dt_2022_10_18_2006_50_12_e_dt_2022_10_18_2017_34_39):13647
delta_WT:
[17281, 17913, 25834, 28058, 23350]
[1.0, 1.0365719576413401, 1.4949366356113651, 1.6236328916150686, 1.3511949539957178]
delta_ES:
[900625, 838622, 1355246, 1746398, 1402100]
[1.0, 0.9311555863983345, 1.5047839000693963, 1.939095628036086, 1.5568077723802916]
delta_ES_APP:
[603757, 473410, 856335, 1331543, 1058711]
[1.0, 0.7841068509350616, 1.418343803881363, 2.205428674118892, 1.7535382612541137]
interpolation result(popt_calc):
[-4.67477751e-04 -2.49584449e-06  5.